In [ ]:
#import libraries
import requests
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
import json
import praw 
from datetime import datetime, timezone

In [ ]:
# Store bearer key in public variable
# API key can get obtained on website : https://developer.themoviedb.org/docs/getting-started

BEARER_KEY = "INSERT KEY"

In [3]:
# Discover Endpoint URL for TMDB Api
url = "https://api.themoviedb.org/3/discover/movie"

headers = {
    "accept": "application/json",
    "Authorization": BEARER_KEY # Replace with your actual API key
}


# Params to filter movies from 1980 to 2010 in english, sorted by popularity
params = {
    "include_adult": "false",
    "include_video": "false",
    "language": "en-US",
    "with_original_language": "en",  
    "sort_by": "popularity.desc",  
    "primary_release_date.gte": "1980-01-01",  
    "primary_release_date.lte": "2009-12-31"   
}

# Empty list to store movies from all pages
all_movies = []

# Loop through pages
# One page only contains 20 movies 
for page_num in range(1, 3):  
    params["page"] = page_num  
    
    # Send the request for the current page
    response = requests.get(url, headers=headers, params=params)
    
    # Check for successful response
    if response.status_code == 200:
        data = response.json()
        movies = data.get("results", [])

        # Append the movies from the current page to the all_movies list
        all_movies.extend(movies)
        
    else:
        # Error output if response is not 200 for a page
        print(f"Error on Page {page_num}: {response.status_code} - {response.text}")


# Extract the date range from params
start_date = params["primary_release_date.gte"]
end_date = params["primary_release_date.lte"]

# Mostly for debugging purpose
# Print the top-rated movies from all pages
print(f"\nTop Rated Movies from {start_date} to {end_date}")
for i, movie in enumerate(all_movies[:10], start=1):  # Show top 10 movies from all pages
    print(f"{i}. {movie['title']} ({movie['release_date']}) - Popularity: {movie['popularity']}")


Top Rated Movies from 1980-01-01 to 2009-12-31
1. Raiders of the Lost Ark (1981-06-12) - Popularity: 134.219
2. The Chronicles of Narnia: The Lion, the Witch and the Wardrobe (2005-12-07) - Popularity: 99.847
3. xXx (2002-08-09) - Popularity: 77.046
4. The Shawshank Redemption (1994-09-23) - Popularity: 72.639
5. Titanic (1997-11-18) - Popularity: 68.694
6. Pirates of the Caribbean: The Curse of the Black Pearl (2003-07-09) - Popularity: 47.678
7. The Dark Knight (2008-07-16) - Popularity: 66.683
8. The Lord of the Rings: The Return of the King (2003-12-17) - Popularity: 62.329
9. Harry Potter and the Philosopher's Stone (2001-11-16) - Popularity: 61.902
10. The Lion King (1994-06-15) - Popularity: 61.013


In [4]:
# Extract movie ids to feed them into movie endpoint of TMDB API
movie_ids = [movie['id'] for movie in all_movies]
# Check if the length 
print(len(movie_ids))

40


In [5]:
# Movie Endpoint URL for TMDB Api
base_url = "https://api.themoviedb.org/3/movie/"

headers = {
    "accept": "application/json",
    "Authorization": BEARER_KEY
}

# List to store movie details
movies_data = []

# Loop through movie IDs to fetch movie details
for movie_id in movie_ids:
    url = f"{base_url}{movie_id}?language=en-US"  # Build the full URL
    
    # Send the request for the current page
    response = requests.get(url, headers=headers)
    
    # Check if response is successful
    if response.status_code == 200:
        movie_details = response.json()
        
        # Store relevant details
        movie_info = {
            "id": movie_details.get("id"),
            "title": movie_details.get("title"),
            "overview": movie_details.get("overview"),
            "release_date": movie_details.get("release_date"),
            "vote_average": movie_details.get("vote_average"),
            "vote_count": movie_details.get("vote_count"),
            "popularity": movie_details.get("popularity"),
            "genres": [genre["name"] for genre in movie_details.get("genres", [])],  
            "budget": movie_details.get("budget"),
            "revenue": movie_details.get("revenue"),
            "runtime": movie_details.get("runtime"),
            "tagline": movie_details.get("tagline"),
            "production_companies": [company["name"] for company in movie_details.get("production_companies", [])],  
            "spoken_languages": [lang["name"] for lang in movie_details.get("spoken_languages", [])]  
        }
        
        # Append movie info to the list
        movies_data.append(movie_info)
    else:
        # If response was not 200
        print(f"Failed to fetch details for movie ID {movie_id}: {response.status_code}")

# Create a DataFrame from the list of movies data
movies_df = pd.DataFrame(movies_data)

In [6]:
movies_df.head(5)

,id,title,overview,release_date,vote_average,vote_count,popularity,genres,budget,revenue,runtime,tagline,production_companies,spoken_languages
0,85,Raiders of the Lost Ark,When Dr. Indiana Jones – the tweed-suited prof...,1981-06-12,7.921,12694,134.219,"[Adventure, Action]",18000000,389925971,115,"If adventure has a name, it must be Indiana Jo...","[Paramount Pictures, Lucasfilm Ltd.]","[English, Español, Deutsch, עִבְרִית, العربية, ]"
1,411,"The Chronicles of Narnia: The Lion, the Witch ...","Siblings Lucy, Edmund, Susan and Peter step th...",2005-12-07,7.135,10790,99.847,"[Adventure, Family, Fantasy]",180000000,745013115,143,The beloved masterpiece comes to life.,"[Walt Disney Pictures, Walden Media, C.S. Lewi...","[English, Deutsch]"
2,7451,xXx,Xander Cage is your standard adrenaline junkie...,2002-08-09,5.900,4354,77.046,"[Action, Adventure, Thriller, Crime]",70000000,277448382,124,A new breed of secret agent.,"[Columbia Pictures, Original Film, Revolution ...","[Český, English, Deutsch, Español, Pусский]"
3,278,The Shawshank Redemption,Imprisoned in the 1940s for the double murder ...,1994-09-23,8.700,27769,72.639,"[Drama, Crime]",25000000,28341469,142,Fear can hold you prisoner. Hope can set you f...,[Castle Rock Entertainment],[English]
4,597,Titanic,101-year-old Rose DeWitt Bukater tells the sto...,1997-11-18,7.907,25720,68.694,"[Drama, Romance]",200000000,2264162353,194,Nothing on Earth could come between them.,"[Paramount Pictures, 20th Century Fox, Lightst...","[English, Français, Deutsch, svenska, Italiano..."


In [7]:
# Create a list of dictionaries with 'title' and extracted 'year'
# This list will be used for scraping websites for the movie ratings, year had to be included since some movies have more than just one version
movie_list = [{"title": title, "year": release_date[:4]} for title, release_date in zip(movies_df["title"], movies_df["release_date"])]

# Print the result for debugging
print(movie_list)

[{'title': 'Raiders of the Lost Ark', 'year': '1981'}, {'title': 'The Chronicles of Narnia: The Lion, the Witch and the Wardrobe', 'year': '2005'}, {'title': 'xXx', 'year': '2002'}, {'title': 'The Shawshank Redemption', 'year': '1994'}, {'title': 'Titanic', 'year': '1997'}, {'title': 'Pirates of the Caribbean: The Curse of the Black Pearl', 'year': '2003'}, {'title': 'The Dark Knight', 'year': '2008'}, {'title': 'The Lord of the Rings: The Return of the King', 'year': '2003'}, {'title': "Harry Potter and the Philosopher's Stone", 'year': '2001'}, {'title': 'The Lion King', 'year': '1994'}, {'title': 'The Lord of the Rings: The Fellowship of the Ring', 'year': '2001'}, {'title': 'Fight Club', 'year': '1999'}, {'title': 'Avatar', 'year': '2009'}, {'title': 'The Abyss', 'year': '1989'}, {'title': 'Ratatouille', 'year': '2007'}, {'title': 'The Matrix', 'year': '1999'}, {'title': 'Gladiator', 'year': '2000'}, {'title': 'Twilight', 'year': '2008'}, {'title': 'Shrek', 'year': '2001'}, {'title

In [17]:
# Initialize the Chrome WebDriver for Rotten Tomatoes
driver = webdriver.Chrome() 

# Define the URL of Rotten Tomatoes
driver_url = "https://www.rottentomatoes.com/"

# Open the URL in the browser
driver.get(driver_url)

In [18]:
# Locate and click the "Accept Cookies" button 
driver.find_element(By.XPATH,'//*[@id="onetrust-accept-btn-handler"]').click()

# Pause execution for 2 seconds to allow the page to process the action
time.sleep(2)

# Zoom out to avoid ads covering search bar or other content
driver.execute_script("document.body.style.zoom='60%'")

In [19]:
# Create an empty list to store data
# Name was given to avoid too many "movie" lists
blueberry_list = []

# Create function to search for a movie title out of the list of dicts, open the movie and find the rating for the searched movie
def search_and_select_movie(movie_title, movie_year):
    # Initialize variables to avoid UnboundLocalError
    audience_score = "N/A"
    genres = "N/A"
    box_office = "N/A"

    try:  
        # Locate the search bar
        search_bar = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '//input[@data-qa="search-input"]'))
        )
        search_bar.clear()  # Clear any previous text
        search_bar.send_keys(movie_title)  # Enter the movie title
        search_bar.send_keys(Keys.RETURN)  # Press Enter to search

        # Wait for the search results page to load
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '//search-page-media-row'))
        )

        time.sleep(3)
        driver.execute_script("document.body.style.zoom='60%'")

        # Find the movie with the specific year
        movie_xpath = f'//search-page-media-row[@releaseyear="{movie_year}"]'
        try:
            movie_item = WebDriverWait(driver, 5).until(
                EC.presence_of_element_located((By.XPATH, movie_xpath))
            )
        except:
            # If movie was not found
            print(f"Movie not found: {movie_title} ({movie_year})")
            blueberry_list.append({
                "Title": movie_title,
                "Year": movie_year,
                "Audience Score": "N/A",
                "Genres": "N/A",
                "Box Office": "N/A"
            })
            return  # Skip to the next movie

        # Click the movie title link
        title_link = movie_item.find_element(By.XPATH, './/a[@data-qa="info-name"]')
        title_link.click()

        # Debugging 
        print(f"Clicked on '{movie_title} ({movie_year})'.")

        # Resize and scroll to avoid ads covering content
        driver.execute_script("document.body.style.zoom='60%'")

        # Click audience scorecard to open the detailed view
        audience_score_element = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//rt-text[@slot='audienceScore']"))
        )
        audience_score_element.click()

        # Wait for detailed ratings
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, "//rt-text[@slot='audienceAverageRating']"))
        )

        # Fetch audience score
        audience_score = driver.find_element(By.XPATH, "//rt-text[@slot='audienceAverageRating']").text

        # Extract genres
        # Genres were extracted to compare them to tmdb genres, in the end we chose tmdb genres due to mostly overlapping information
        genre_elements = driver.find_elements(By.XPATH, "//div[@class='category-wrap']//rt-link[contains(@href, '/browse/movies_in_theaters/genres:')]")
        genres = ", ".join([genre.get_attribute("textContent").strip() for genre in genre_elements if genre.get_attribute("textContent").strip()])

        # Extract Box Office Earnings
        # Same reason as for genres, rotten tomatoes box office was collected for the us market and tmdb included worldwide box office number
        try:
            box_office = driver.find_element(By.XPATH, "//div[@class='category-wrap']//rt-text[contains(text(), '$')]").text
        except:
            box_office = "N/A"

        print(f"Movie: {movie_title} ({movie_year})")
        print(f"Audience Score: {audience_score}")
        print(f"Genres: {genres}")
        print(f"Box Office: {box_office}\n")

    # Debugging
    except Exception as e:
        print(f"Error while scraping {movie_title} ({movie_year}): {e}")

    # Append data to list even if movie was not found
    blueberry_list.append({
        "Title": movie_title,
        "Year": movie_year,
        "Audience Score": audience_score,
        "Genres": genres,
        "Box Office": box_office
    })

# Apply funtion to movie_list dict
for movie in movie_list:
    search_and_select_movie(movie["title"], movie["year"])

# Convert the list of dictionaries into a DataFrame
rotten_df = pd.DataFrame(blueberry_list)

# Display the DataFrame
print(rotten_df)

Clicked on 'Raiders of the Lost Ark (1981)'.
Movie: Raiders of the Lost Ark (1981)
Audience Score: 4.5 out of 5 Rating
Genres: Adventure, Action
Box Office: $248.2M

Clicked on 'The Chronicles of Narnia: The Lion, the Witch and the Wardrobe (2005)'.
Movie: The Chronicles of Narnia: The Lion, the Witch and the Wardrobe (2005)
Audience Score: 3.1 out of 5 Rating
Genres: Kids & Family, Fantasy, Adventure
Box Office: $291.7M

Movie not found: xXx (2002)
Clicked on 'The Shawshank Redemption (1994)'.
Movie: The Shawshank Redemption (1994)
Audience Score: 4.7 out of 5 Rating
Genres: Drama
Box Office: $27.3M

Clicked on 'Titanic (1997)'.
Movie: Titanic (1997)
Audience Score: 3.3 out of 5 Rating
Genres: History, Drama, Romance
Box Office: $658.8M

Clicked on 'Pirates of the Caribbean: The Curse of the Black Pearl (2003)'.
Movie: Pirates of the Caribbean: The Curse of the Black Pearl (2003)
Audience Score: 3.9 out of 5 Rating
Genres: Adventure, Action, Fantasy
Box Office: $305.4M

Clicked on 'Th

In [20]:
rotten_df.head(5)

,Title,Year,Audience Score,Genres,Box Office
0,Raiders of the Lost Ark,1981,4.5 out of 5 Rating,"Adventure, Action",$248.2M
1,"The Chronicles of Narnia: The Lion, the Witch ...",2005,3.1 out of 5 Rating,"Kids & Family, Fantasy, Adventure",$291.7M
2,xXx,2002,N/A,N/A,N/A
3,The Shawshank Redemption,1994,4.7 out of 5 Rating,Drama,$27.3M
4,Titanic,1997,3.3 out of 5 Rating,"History, Drama, Romance",$658.8M


In [21]:
# Load Reddit credentials for Reddit Api
with open("reddit_credentials.json", "r") as file:
    creds = json.load(file)

# Using praw library to access content
reddit = praw.Reddit(
    client_id=creds["client_id"],
    client_secret=creds["client_secret"],
    user_agent=creds["user_agent"],
    username=creds["username"],
    password=creds["password"]
)

print(f"Logged in as: {reddit.user.me()}")

Version 7.7.1 of praw is outdated. Version 7.8.1 was released Friday October 25, 2024.


Logged in as: mariuskiefer


In [23]:
# Create empty list to store data
all_posts = []

# Function to search Reddit for a movie (ensuring unique users)
# Sometimes one user is very active about a movie. To avoid misinterpretation about the sentiment about a movie, we only allowed unique users 
def search_reddit(movie_title, movie_year):
    search_query = f"{movie_title} {movie_year}"
    posts = reddit.subreddit("all").search(search_query, limit=200, sort="new")  # Retrieve more to ensure uniqueness

    results = []
    seen_authors = set()  # Track unique authors

    for post in posts:
        author_name = post.author.name if post.author else "[Deleted]"

        if author_name in seen_authors:
            continue  # Skip if author is already added

        post_date = datetime.fromtimestamp(post.created_utc, tz=timezone.utc).strftime('%Y-%m-%d')
        results.append({
            "movie_title": movie_title,
            "movie_year": movie_year,
            "reddit_title": post.title,
            "author": author_name,
            "subreddit": post.subreddit.display_name,
            "url": post.url,
            "score": post.score,
            "comments": post.num_comments,
            "date": post_date
        })

        seen_authors.add(author_name)  # Mark author as processed

        if len(results) >= 50:  # Stop once we have 100 unique users
            break

    return results

# Iterate through the list of movies and search Reddit
for movie in movie_list:
    title = movie["title"]
    year = movie["year"]
    print(f"\nSearching Reddit for '{title} ({year})'...")
    
    posts = search_reddit(title, year)

    if not posts:
        print(f"No results found for {title} ({year}).")
        continue

    all_posts.extend(posts)  # Store results in the list

# Convert to DataFrame
reddit_df = pd.DataFrame(all_posts)



Searching Reddit for 'Raiders of the Lost Ark (1981)'...

Searching Reddit for 'The Chronicles of Narnia: The Lion, the Witch and the Wardrobe (2005)'...

Searching Reddit for 'xXx (2002)'...

Searching Reddit for 'The Shawshank Redemption (1994)'...

Searching Reddit for 'Titanic (1997)'...

Searching Reddit for 'Pirates of the Caribbean: The Curse of the Black Pearl (2003)'...

Searching Reddit for 'The Dark Knight (2008)'...

Searching Reddit for 'The Lord of the Rings: The Return of the King (2003)'...

Searching Reddit for 'Harry Potter and the Philosopher's Stone (2001)'...

Searching Reddit for 'The Lion King (1994)'...

Searching Reddit for 'The Lord of the Rings: The Fellowship of the Ring (2001)'...

Searching Reddit for 'Fight Club (1999)'...

Searching Reddit for 'Avatar (2009)'...

Searching Reddit for 'The Abyss (1989)'...

Searching Reddit for 'Ratatouille (2007)'...

Searching Reddit for 'The Matrix (1999)'...

Searching Reddit for 'Gladiator (2000)'...

Searching Redd

In [24]:
# Initialize the Chrome WebDriver for Letterbox
driver = webdriver.Chrome()

# Open Driver with Letterbox URL
driver.get("https://letterboxd.com/")

In [25]:
# Click the "Consent" button on Letterboxd
try:
    consent_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, "//button[contains(@class, 'fc-cta-consent')]"))
    )
    consent_button.click()
    print("Clicked Consent button.")
except:
    print("Consent button not found or already accepted.")

Clicked Consent button.


In [26]:
# NOTE: When Driver opens the Movie page Ads occur that hide important content that we try to scrape. 
# In order to work around this problem we allow the ads to load, using a timer. We then proceed with zooming out to ensure the content is visible.
# The line of code with the timer is highlighted with a comment in capslock. Timer needs to be adjusted if the scraping mechanism does not work with initial timer settings.

# Create an empty list to store data
raspberry_list = []

def search_and_select_movie(movie_title, movie_year):
    try:
        # Replace straight apostrophe with curly one
        # Movies could not be found containing straight apostrophes
        movie_title = movie_title.replace("'", "’")

        # Locate the search bar on Letterboxd
        search_bar = WebDriverWait(driver, 30).until(
            EC.presence_of_element_located((By.ID, "search-q"))
        )
        search_bar.clear()
        search_bar.send_keys(movie_title)
        search_bar.send_keys(Keys.RETURN)

        # Wait for search results to load
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, "results"))
        )

        # Find all movie results
        movie_results = driver.find_elements(By.XPATH, "//ul[contains(@class, 'results')]/li")

        for movie in movie_results:
            try:
                # Locate the movie title within each search result
                title_text = movie.find_element(By.XPATH, ".//span[@class='film-title-wrapper']/a").text.strip()
                year_text = movie.find_element(By.XPATH, ".//small[@class='metadata']/a").text.strip()

                if title_text.lower() == movie_title.lower() and year_text == movie_year:
                    movie_link = movie.find_element(By.XPATH, ".//span[@class='film-title-wrapper']/a").get_attribute("href")
                    driver.get(movie_link)  # Navigate to the movie page
                    print(f"Navigated to: {movie_title} ({movie_year}) - {movie_link}")
                    break
            except Exception as e:
                print(f"Error processing movie result: {e}")
                continue

        # Fetch the rating
        try:

            # ADJUST (INCREASE) THE TIMER IF ADS DIDNT HAVE ENOUGH TIME TO LOAD
            # GET RID OF TIMER IF SYSTEM IS VERY FAST SO THE CONTENT LOADS BEFORE ADS CAN EVEN POP UP
            time.sleep(3)
            driver.execute_script("document.body.style.zoom='60%'")

            rating_element = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, "//a[contains(@class, 'tooltip display-rating')]"))
            )
            rating = rating_element.text.strip()
        except:
            rating = "N/A"
        
        print(f"Movie: {movie_title} ({movie_year}) - Rating: {rating}")

        # Wait for the movie page to load
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, "film-poster"))
        )

        # Store data in a dictionary
        info_dict = {
            "Title": movie_title,
            "Year": movie_year,
            "Audience Score": rating,
        }
        raspberry_list.append(info_dict)

    except Exception as e:
        print(f"Error while scraping {movie_title} ({movie_year}): {e}")

# Loop through movie list
for movie in movie_list:
    search_and_select_movie(movie["title"], movie["year"])

# Convert the list of dictionaries into a DataFrame
letterboxd_df = pd.DataFrame(raspberry_list)

# Display the DataFrame
print(letterboxd_df)

Navigated to: Raiders of the Lost Ark (1981) - https://letterboxd.com/film/raiders-of-the-lost-ark/
Movie: Raiders of the Lost Ark (1981) - Rating: 4.2
Navigated to: The Chronicles of Narnia: The Lion, the Witch and the Wardrobe (2005) - https://letterboxd.com/film/the-chronicles-of-narnia-the-lion-the-witch-and-the-wardrobe/
Movie: The Chronicles of Narnia: The Lion, the Witch and the Wardrobe (2005) - Rating: 3.6
Navigated to: xXx (2002) - https://letterboxd.com/film/xxx/
Movie: xXx (2002) - Rating: 2.5
Navigated to: The Shawshank Redemption (1994) - https://letterboxd.com/film/the-shawshank-redemption/
Movie: The Shawshank Redemption (1994) - Rating: 4.6
Navigated to: Titanic (1997) - https://letterboxd.com/film/titanic-1997/
Movie: Titanic (1997) - Rating: 3.8
Navigated to: Pirates of the Caribbean: The Curse of the Black Pearl (2003) - https://letterboxd.com/film/pirates-of-the-caribbean-the-curse-of-the-black-pearl/
Movie: Pirates of the Caribbean: The Curse of the Black Pearl (2

In [27]:
# Extract the first number from "Audience Score" and convert to float
rotten_df["Audience Score Cleaned"] = rotten_df["Audience Score"].str.extract(r"(\d+(\.\d+)?)")[0].astype(float)

# Drop the old column
# Year is no longer needed
rotten_df = rotten_df.drop(columns=["Audience Score"])
rotten_df = rotten_df.drop(columns=["Year"])

rotten_df.head(5)

,Title,Genres,Box Office,Audience Score Cleaned
0,Raiders of the Lost Ark,"Adventure, Action",$248.2M,4.5
1,"The Chronicles of Narnia: The Lion, the Witch ...","Kids & Family, Fantasy, Adventure",$291.7M,3.1
2,xXx,N/A,N/A,NaN
3,The Shawshank Redemption,Drama,$27.3M,4.7
4,Titanic,"History, Drama, Romance",$658.8M,3.3


In [28]:
# Year is no longer needed
letterboxd_df = letterboxd_df.drop(columns=["Year"])

letterboxd_df.head(5)

,Title,Audience Score
0,Raiders of the Lost Ark,4.2
1,"The Chronicles of Narnia: The Lion, the Witch ...",3.6
2,xXx,2.5
3,The Shawshank Redemption,4.6
4,Titanic,3.8


In [29]:
# Getting rid of unused columns
movies_df = movies_df.drop(columns=["overview", "budget", "tagline", "production_companies", "spoken_languages"])

movies_df.head(5)

,id,title,release_date,vote_average,vote_count,popularity,genres,revenue,runtime
0,85,Raiders of the Lost Ark,1981-06-12,7.921,12694,134.219,"[Adventure, Action]",389925971,115
1,411,"The Chronicles of Narnia: The Lion, the Witch ...",2005-12-07,7.135,10790,99.847,"[Adventure, Family, Fantasy]",745013115,143
2,7451,xXx,2002-08-09,5.900,4354,77.046,"[Action, Adventure, Thriller, Crime]",277448382,124
3,278,The Shawshank Redemption,1994-09-23,8.700,27769,72.639,"[Drama, Crime]",28341469,142
4,597,Titanic,1997-11-18,7.907,25720,68.694,"[Drama, Romance]",2264162353,194


In [30]:
# Getting rid of unused columns
reddit_df = reddit_df.drop(columns=["movie_year", "reddit_title", "subreddit", "author", "url"])


reddit_df.head(5)

,movie_title,score,comments,date
0,Raiders of the Lost Ark,1,1,2025-02-26
1,Raiders of the Lost Ark,3,1,2025-02-26
2,Raiders of the Lost Ark,0,8,2025-02-26
3,Raiders of the Lost Ark,1,0,2025-02-26
4,Raiders of the Lost Ark,1,0,2025-02-26


In [31]:
# Sort posts in Week1, Week2-3 and Week3+ based on recency 
# Ensure 'date' column is in datetime format
reddit_df['date'] = pd.to_datetime(reddit_df['date'], format='%Y-%m-%d')

# Calculate post age in days from today
today = pd.to_datetime("today").normalize()
reddit_df['days_old'] = (today - reddit_df['date']).dt.days

# Create the new period columns
reddit_df['Week1'] = reddit_df['days_old'].between(0, 7).astype(int)  # Posts in the weeks
reddit_df['Week2-3'] = reddit_df['days_old'].between(8, 21).astype(int) # Posts between 2-3 weeks
reddit_df['Week3+'] = (reddit_df['days_old'] > 21).astype(int)           # Older than 3 weeks

# Preserve index before grouping
reddit_df = reddit_df.set_index("movie_title")  

# Perform groupby to aggregate at the movie level while keeping the original index
aggregated_df = reddit_df.groupby(level=0).agg({
    'score': 'sum',       # Sum all scores for each movie
    'comments': 'sum',    # Sum all comments for each movie
    'Week1': 'sum',     # Count posts in Week1
    'Week2-3': 'sum',     # Count posts in Week2-3
    'Week3+': 'sum'       # Count posts in Week3+
})

# Restore the original index
aggregated_df = aggregated_df.reset_index()

aggregated_df.head(10)

,movie_title,score,comments,Week1,Week2-3,Week3+
0,Avatar,18180,1975,17,23,10
1,Coraline,12792,705,2,4,44
2,Fight Club,2347,617,50,0,0
3,Finding Nemo,1717,754,50,0,0
4,Forrest Gump,2106,1861,50,0,0
5,Gladiator,887,371,44,6,0
6,Harry Potter and the Chamber of Secrets,736,247,50,0,0
7,Harry Potter and the Goblet of Fire,764,345,50,0,0
8,Harry Potter and the Philosopher's Stone,672,297,50,0,0
9,Harry Potter and the Prisoner of Azkaban,1903,427,50,0,0


In [32]:
# Relevance Score to indicate if a movie is still relevant to people (on reddit)
aggregated_df["Relevance_Score"] = (
    aggregated_df["Week1"] * 3 + # weighted the most for relevance
    aggregated_df["Week2-3"] * 2 + 
    aggregated_df["Week3+"] * 1
) / 30 # To scale relevante from 0 to 5

aggregated_df.head(10)

,movie_title,score,comments,Week1,Week2-3,Week3+,Relevance_Score
0,Avatar,18180,1975,17,23,10,3.566667
1,Coraline,12792,705,2,4,44,1.933333
2,Fight Club,2347,617,50,0,0,5.000000
3,Finding Nemo,1717,754,50,0,0,5.000000
4,Forrest Gump,2106,1861,50,0,0,5.000000
5,Gladiator,887,371,44,6,0,4.800000
6,Harry Potter and the Chamber of Secrets,736,247,50,0,0,5.000000
7,Harry Potter and the Goblet of Fire,764,345,50,0,0,5.000000
8,Harry Potter and the Philosopher's Stone,672,297,50,0,0,5.000000
9,Harry Potter and the Prisoner of Azkaban,1903,427,50,0,0,5.000000


In [33]:
# Sort movies alphabetically to merge them in the next step
movies_df = movies_df.sort_values(by="title", ascending=True).reset_index(drop=True)

rotten_df = rotten_df.sort_values(by="Title", ascending=True).reset_index(drop=True)

letterboxd_df = letterboxd_df.sort_values(by="Title", ascending=True).reset_index(drop=True)


In [34]:

# Merge the three DataFrames on the index
merged_df = pd.concat([rotten_df, movies_df, aggregated_df, letterboxd_df], axis=1)

# Getting rid of redunant columns
merged_df = merged_df.drop(columns=["title", "movie_title"])
merged_df = merged_df.iloc[:, ~merged_df.columns.duplicated(keep='first')]
merged_df = merged_df.drop(columns=["Genres"])

# Renaming columns
merged_df.rename(columns={
    'Audience Score Cleaned': 'Rating Rotten',
    'Audience Score': 'Rating Letterbox'
}, inplace=True)

# Chaning the TMDB Rating to a 0 to 5 Rating
merged_df['Rating TMDB'] = merged_df['vote_average'] / 2
merged_df = merged_df.drop(columns=["vote_average"])
merged_df = merged_df.drop(columns=["vote_count"])

# Put Rotten Rating to the end so it is easy to compare all Ratings next to each other
cols = [col for col in merged_df.columns if col != 'Rating Rotten'] + ['Rating Rotten']
merged_df = merged_df[cols]

merged_df.head(10)

,Title,Box Office,id,release_date,popularity,genres,revenue,runtime,score,comments,Week1,Week2-3,Week3+,Relevance_Score,Rating Letterbox,Rating TMDB,Rating Rotten
0,Avatar,$23.5M,19995,2009-12-15,51.743,"[Action, Adventure, Fantasy, Science Fiction]",2923706026,162,18180,1975,17,23,10,3.566667,3.6,3.7935,4.1
1,Coraline,$94.2M,14836,2009-02-05,47.912,"[Animation, Family, Fantasy]",185860104,100,12792,705,2,4,44,1.933333,4.2,3.9500,3.8
2,Fight Club,$37.0M,550,1999-10-15,51.892,[Drama],100853753,139,2347,617,50,0,0,5.000000,4.3,4.2190,4.5
3,Finding Nemo,N/A,12,2003-05-30,39.322,"[Animation, Family]",940335536,100,1717,754,50,0,0,5.000000,4.0,3.9085,3.8
4,Forrest Gump,$330.3M,13,1994-06-23,41.146,"[Comedy, Drama, Romance]",677387716,142,2106,1861,50,0,0,5.000000,4.1,4.2340,4.1
5,Gladiator,$187.7M,98,2000-05-04,50.518,"[Action, Drama, Adventure]",465361176,155,887,371,44,6,0,4.800000,4.1,4.1090,3.8
6,Harry Potter and the Chamber of Secrets,$262.0M,672,2002-11-13,45.069,"[Adventure, Fantasy]",876688482,161,736,247,50,0,0,5.000000,3.6,3.8500,4.0
7,Harry Potter and the Goblet of Fire,$290.0M,674,2005-11-16,43.086,"[Adventure, Fantasy]",895921036,157,764,345,50,0,0,5.000000,3.8,3.9000,3.4
8,Harry Potter and the Philosopher's Stone,$317.9M,671,2001-11-16,61.902,"[Adventure, Fantasy]",976475550,152,672,297,50,0,0,5.000000,3.8,3.9500,4.1
9,Harry Potter and the Prisoner of Azkaban,$249.4M,673,2004-05-31,46.056,"[Adventure, Fantasy]",789804554,141,1903,427,50,0,0,5.000000,4.1,4.0000,4.1


In [35]:
# Drop unused columns
merged_df = merged_df.drop(columns=["id", "Box Office", "runtime"])


In [36]:
# Renaming columns before extracting df to csv
merged_df.rename(columns={
                "release_date": "Release Date",
                "popularity": "Popularity",
                "genres": "Genres",
                "revenue": "Revenue",
                "score": "Reddit Score",
                "comments": "Reddit Comments",
                "Week1": "N Reddit Posts Week1",
                "Week2-3": "N Reddit Posts Week2-3",
                "Week3+": "N Reddit Posts Week3+",
                "Relevance_Score": "Reddit Relevance Score",
                
                # Add more columns as needed
            }, inplace=True)


In [37]:
merged_df.head(5)

,Title,Release Date,Popularity,Genres,Revenue,Reddit Score,Reddit Comments,N Reddit Posts Week1,N Reddit Posts Week2-3,N Reddit Posts Week3+,Reddit Relevance Score,Rating Letterbox,Rating TMDB,Rating Rotten
0,Avatar,2009-12-15,51.743,"[Action, Adventure, Fantasy, Science Fiction]",2923706026,18180,1975,17,23,10,3.566667,3.6,3.7935,4.1
1,Coraline,2009-02-05,47.912,"[Animation, Family, Fantasy]",185860104,12792,705,2,4,44,1.933333,4.2,3.9500,3.8
2,Fight Club,1999-10-15,51.892,[Drama],100853753,2347,617,50,0,0,5.000000,4.3,4.2190,4.5
3,Finding Nemo,2003-05-30,39.322,"[Animation, Family]",940335536,1717,754,50,0,0,5.000000,4.0,3.9085,3.8
4,Forrest Gump,1994-06-23,41.146,"[Comedy, Drama, Romance]",677387716,2106,1861,50,0,0,5.000000,4.1,4.2340,4.1


In [ ]:
# Convert df to csv
merged_df.to_csv("API_Scraping_DATA.csv", index=False)